# Cosmosdb connection testing

## Azure python documentation
https://learn.microsoft.com/en-us/python/api/azure-cosmos/azure.cosmos?preserve-view=true&view=azure-python


In [6]:
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
import os
from dotenv import load_dotenv
from azure.cosmos import CosmosClient, PartitionKey

load_dotenv('../../shared/.env')

True

In [7]:
# database env
URL = os.getenv('COSMOSDB_ENDPOINT')  
MASTER_KEY = os.getenv('COSMOSDB_KEY')

In [8]:
# init client
client = cosmos_client.CosmosClient(URL, {'masterKey': MASTER_KEY}, user_agent="Andy", user_agent_overwrite=True)


In [ ]:
# Create a database (if it doesn't exist)
database_name = 'SampleDatabase'

try:
    database = client.create_database_if_not_exists(id=database_name)
except exceptions.CosmosResourceExistsError:
    database = client.get_database_client(database_name)

# Create a container (if it doesn't exist)
container_name = 'SampleContainer'
try:
    container = database.create_container_if_not_exists(
        id=container_name,
        partition_key=PartitionKey(path='/partitionKey'),
        offer_throughput=400
    )
except exceptions.CosmosResourceExistsError:
    container = database.get_container_client(container_name)

In [ ]:
# insert something in the db
